In [2]:
from fastai.vision import *

In [90]:
class CSVLogger(LearnerCallback):
    "A `LearnerCallback` that saves history of metrics while training `learn` into CSV `filename`."
    def __init__(self, learn:Learner, filename: str = 'history', refresh: bool = True): 
        super().__init__(learn)
        self.filename,self.path, self.refresh = filename,self.learn.path/f'{filename}.csv', refresh

    def read_logged_file(self):  
        "Read the content of saved file"
        return pd.read_csv(self.path)

    def on_train_begin(self, **kwargs: Any) -> None:
        "Prepare file with metric names."
        self.path.parent.mkdir(parents=True, exist_ok=True)      
        self.file = self.path.open('w') if self.refresh else self.path.open('a')
        self.file.write(','.join(self.learn.recorder.names) + '\n')
        
    def on_epoch_end(self, epoch: int, smooth_loss: Tensor, last_metrics: MetricsList, **kwargs: Any) -> bool:
        "Add a line with `epoch` number, `smooth_loss` and `last_metrics`."
        last_metrics = ifnone(last_metrics, [])
        stats = [str(stat) if isinstance(stat, int) else f'{stat:.6f}'
                 for name, stat in zip(self.learn.recorder.names, [epoch, smooth_loss] + last_metrics)]
        str_stats = ','.join(stats)
        self.file.write(str_stats + '\n')

    def on_train_end(self, **kwargs: Any) -> None:  
        "Close the file."
        self.file.close()

In [91]:
path = untar_data(URLs.MNIST_TINY)
data = ImageDataBunch.from_folder(path)
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy, error_rate],
                callback_fns=[partial(CSVLogger, refresh=False)])

In [92]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.653771,0.591294,0.525036,0.474964,00:00


In [93]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.538662,0.409373,0.927039,0.072961,00:00


In [94]:
learn.csv_logger.read_logged_file()

,epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0,0.653771,0.591294,0.525036,0.474964,NaN
1,epoch,train_loss,valid_loss,accuracy,error_rate,time
2,0,0.538662,0.409373,0.927039,0.072961,NaN
